In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp augmentation

# Data augmentation

> Functions to augment the user's dataset with information from official sources.

In [ ]:
#hide
from nbdev.showdoc import show_doc

`gingado` provides data augmentation functionalities that can help users to augment their datasets with a time series dimension. This can be done both on a stand-alone basis as the user incorporates new data on top of the original dataset, or as part of a `scikit-learn` [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that also includes other steps like data transformation and model estimation.

## Data augmentation with SDMX

The **S**tatistical **D**ata and **M**etadata e**X**change (SDMX) is an ISO standard comprising:
* technical standards
* statistical guidelines, including cross-domain concepts and codelists
* an IT architecture and tools

SDMX is sponsored by the Bank for International Settlements, European Central Bank, Eurostat, International Monetary Fund, Organisation for Economic Co-operation and Development, United Nations, and World Bank Group.

More information about the SDMX is available on its [webpage](http://sdmx.org).

In [ ]:
#hide
#export
import pandas as pd
import pandasdmx as sdmx
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_selection import VarianceThreshold

class AugmentSDMX(BaseEstimator, TransformerMixin):

    InputIndexMessage = "The dataset to be augmented must have a row index with the date/time information"
    def _format_string(self):
        return "%Y-%m-%d" if self.data_freq_ == 'D' else "%Y-%m" if self.data_freq_ == 'M' else "%Y"
    
    def _get_dates(self):
        fstr = self._format_string()
        return {
            "startPeriod": min(self.index_).strftime(fstr),
            "endPeriod": max(self.index_).strftime(fstr),
        }

    def _transform(self, X, training=True):
        data_sdmx = {}
        for source in self.sources.keys():
            src_conn = sdmx.Request(source)
            src_dflows = src_conn.dataflow()
            if self.sources[source] == 'all':
                dflows = {k: v for k, v in src_dflows.dataflow.items()}
            else:
                dflows = {k: v for k, v in src_dflows.dataflow.items() if k in self.sources[source]}
            for dflow in dflows.keys():
                if self.verbose: print(f"Querying data from {source}'s dataflow '{dflow}' - {dflows[dflow].dict()['name']}...")
                try:
                    data = sdmx.to_pandas(src_conn.data(dflow, key=self.keys_, params=self.params_), datetime='TIME_PERIOD')
                except:
                    if self.verbose: print("this dataflow does not have data in the desired frequency and time period.")
                    continue
                data.columns = ['__'.join(col) for col in data.columns.to_flat_index()]
                data_sdmx[source+"__"+dflow] = data
        
        if len(data_sdmx.keys()) is None:
            return X

        df = pd.concat(data_sdmx, axis=1)
        df.columns = ['_'.join(col) for col in df.columns.to_flat_index()]

        if training:
            # test that the dataset `X` has the same dimension as the one
            # used during training, which is an evidence they are the same
            n_samples_in_transform, n_features_in_transform = X.shape
            if n_samples_in_transform != self.n_samples_in_ or n_features_in_transform != self.n_features_in_:
                raise ValueError("The X passed to the transform() method must be compatible with the X used by the fit() method.")
            # during testing, we don't want the possibility of a different
            # set of columns being retained by virtue of different dynamics
            # in both datasets. For example, if a feature is included in the
            # training but during the test dates the variable didn't move, it
            # should not be subject to the test below so that it is still
            # included in the fitted data.
            feat_sel = VarianceThreshold() if self.variance_threshold is None else VarianceThreshold(threshold=self.variance_threshold)
            feat_sel.fit(df)
            self.features_stay_ = df.columns[feat_sel.get_support()]
            self.features_removed_ = df.columns[~feat_sel.get_support()]
            
            df = df.iloc[:, feat_sel.get_support()]
            df.columns = feat_sel.get_feature_names_out()

            df.dropna(axis=0, how='all', inplace=True)
            df.dropna(axis=1, how='all', inplace=True)        

        
        X = pd.merge(left=X, right=df, how='left', left_index=True, right_on='TIME_PERIOD')
        if 'TIME_PERIOD' in X.columns:
            X.drop(columns='TIME_PERIOD', inplace=True)
        if self.propagate_last_known_value:
            X.fillna(method="ffill", inplace=True)
        if training:
            X.index = self.index_
        return X

    def __init__(self, sources={'BIS': 'WS_CBPOL_D'}, variance_threshold=None, propagate_last_known_value=True, verbose=True):
        self.sources = sources
        self.variance_threshold = variance_threshold
        self.propagate_last_known_value = propagate_last_known_value
        self.verbose = verbose

    def fit(self, X, y=None):
        """
        Fits transformer to `X`; `y` is kept as argument for API consistency only.

        Parameters
        ----------
        X : a pandas Series or DataFrame with an index of datetime type
            Input samples.
        y : default=None

        Returns
        -------
        The fitted version of the instance of `AugmentSDMX`, ie after it learned \
        the frequency of the time series in `X`. The possible values fitted on the \
        data are described in: \
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timedelta.resolution_string.html.
        """
        try:
            self.data_freq_ = X.index.to_series().diff().min().resolution_string
        except AttributeError:
            print(self.InputIndexMessage)
            raise
        self.index_ = X.index
        self.keys_ = {'FREQ': self.data_freq_}
        X = self._validate_data(X)

        # this variable below is only included to test for consistency \
        # if `fit` and `transform` are called separately with the same `X`
        self.n_samples_in_ = X.shape[0]

        return self

    def transform(self, X, y=None, training=False):
        check_is_fitted(self)
        self.params_ = self._get_dates()
        idx = X.index
        transf_X = self._transform(X, training=training)
        transf_X.index = idx
        return transf_X

    def fit_transform(self, X, y=None):
        """
        Fit to data, then transform it.
        Fits transformer to `X` and returns a transformed version of `X`. 
        `y` is kept as argument for API consistency only.

        Parameters
        ----------
        X : a pandas Series or DataFrame with an index of datetime type
            Input samples.
        y : default=None
            Target values (None for unsupervised transformations).
        training : True / False. default=False
            The default value ensures that the when this transformer is called \
            by a pipeline
        Returns
        -------
        X_new : ndarray array of shape (n_samples, n_features_new)
            Transformed array.
        """
        
        self.fit(X)
        self.params_ = self._get_dates()
        return self.transform(X, training=True)

In [ ]:
show_doc(AugmentSDMX.fit)

<h4 id="AugmentSDMX.fit" class="doc_header"><code>AugmentSDMX.fit</code><a href="__main__.py#L87" class="source_link" style="float:right">[source]</a></h4>

> <code>AugmentSDMX.fit</code>(**`X`**, **`y`**=*`None`*)

Fits transformer to `X`; `y` is kept as argument for API consistency only.

Parameters
----------
X : a pandas Series or DataFrame with an index of datetime type
    Input samples.
y : default=None

Returns
-------
The fitted version of the instance of [`AugmentSDMX`](/gingado/augmentation.html#AugmentSDMX), ie after it learned         the frequency of the time series in `X`. The possible values fitted on the         data are described in:         https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timedelta.resolution_string.html.

As mentioned above, `gingado`'s transformers are built to be compatible with `scikit-learn`. The code below demonstrates this compatibility.

First, we create the example dataset. In this case, it comprises the daily foreign exchange rate of selected currencies to the Euro. The Brazilian Real (BRL) is chosen for this example as the dependent variable.

In [ ]:
from gingado.utils import load_EURFX_data
from sklearn.model_selection import TimeSeriesSplit

X = load_EURFX_data(keep_contemporaneous_X=True)
y = X.pop('BRL')
# retain only the lagged variables
X = X[X.columns[X.columns.str.contains('_lag_')]]

In [ ]:
X_train, X_test = X.iloc[:-1], X.tail(1)
y_train, y_test = y.iloc[:-1], y.tail(1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4968, 8), (4968,), (1, 8), (1,))

Next, the data augmentation object provided by `gingado` adds more data. In this case, for brevity only one dataflow from one source is listed. If users want to add more SDMX sources, simply add more keys to the dictionary. And if users want data from all dataflows from a given source (provided the keys and parameters such as frequency and dates are matched), the value should be set to `'all'`, as in `{'ECB': ['CISS'], 'BIS': 'all'}`.

In [ ]:
from gingado.augmentation import AugmentSDMX

test_src = {'ECB': ['CISS'], 'BIS': ['WS_CBPOL_D']}

X_train__fit_transform = AugmentSDMX(sources=test_src).fit_transform(X=X_train)
X_train__fit_then_transform = AugmentSDMX(sources=test_src).fit(X=X_train).transform(X=X_train, training=True)

assert X_train__fit_transform.shape == X_train__fit_then_transform.shape

Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...


2022-05-28 01:07:13,775 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from BIS's dataflow 'WS_CBPOL_D' - Policy rates daily...
Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...


2022-05-28 01:08:49,632 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from BIS's dataflow 'WS_CBPOL_D' - Policy rates daily...


`AugmentSDMX` can also be part of a `Pipeline` object, which minimises operational errors during modelling and avoids using testing data during training:

This is the dataset now after this particular augmentation:

In [ ]:
print(f"No of columns: {len(X_train__fit_transform.columns)} {X_train__fit_transform.columns}")
X_train__fit_transform

No of columns: 68 Index(['AUD_lag_1', 'BRL_lag_1', 'CAD_lag_1', 'CHF_lag_1', 'GBP_lag_1',
       'JPY_lag_1', 'SGD_lag_1', 'USD_lag_1',
       'ECB__CISS_D__AT__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__BE__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__CN__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__DE__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__ES__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__FI__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__FR__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__GB__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__IE__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__IT__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__NL__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__PT__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__U2__Z0Z__4F__EC__SS_BM__CON',
       'ECB__CISS_D__U2__Z0Z__4F__EC__SS_CI__IDX',
       'ECB__CISS_D__U2__Z0Z__4F__EC__SS_CIN__IDX',
       'ECB__CISS_D__U2__Z0Z__4F__EC__SS_CO__CON',
       'ECB__CISS_D__U2__Z0Z__4F__E

,AUD_lag_1,BRL_lag_1,CAD_lag_1,CHF_lag_1,GBP_lag_1,JPY_lag_1,SGD_lag_1,USD_lag_1,ECB__CISS_D__AT__Z0Z__4F__EC__SS_CIN__IDX,ECB__CISS_D__BE__Z0Z__4F__EC__SS_CIN__IDX,...,BIS__WS_CBPOL_D_D__RO,BIS__WS_CBPOL_D_D__RS,BIS__WS_CBPOL_D_D__RU,BIS__WS_CBPOL_D_D__SA,BIS__WS_CBPOL_D_D__SE,BIS__WS_CBPOL_D_D__TH,BIS__WS_CBPOL_D_D__TR,BIS__WS_CBPOL_D_D__US,BIS__WS_CBPOL_D_D__XM,BIS__WS_CBPOL_D_D__ZA
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
2003-01-03,1.8554,3.6770,1.6422,1.4528,0.65200,124.40,1.8188,1.0446,0.021899,0.043292,...,NaN,9.5,NaN,NaN,3.75,1.75,44.0,1.250,2.75,13.50
2003-01-06,1.8440,3.6112,1.6264,1.4555,0.65000,124.56,1.8132,1.0392,0.020801,0.039924,...,19.75,9.5,NaN,2.00,3.75,1.75,44.0,1.250,2.75,13.50
2003-01-07,1.8281,3.5145,1.6383,1.4563,0.64950,124.40,1.8210,1.0488,0.019738,0.038084,...,19.75,9.5,NaN,2.00,3.75,1.75,44.0,1.250,2.75,13.50
2003-01-08,1.8160,3.5139,1.6257,1.4565,0.64960,124.82,1.8155,1.0425,0.019947,0.040338,...,19.75,9.5,21.0,2.00,3.75,1.75,44.0,1.250,2.75,13.50
2003-01-09,1.8132,3.4405,1.6231,1.4586,0.64950,124.90,1.8102,1.0377,0.017026,0.040535,...,19.75,9.5,21.0,2.00,3.75,1.75,44.0,1.250,2.75,13.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-20,1.5036,5.2094,1.3490,1.0265,0.84728,134.46,1.4576,1.0525,0.288026,0.189337,...,3.75,2.0,14.0,1.75,0.25,0.50,14.0,0.875,0.00,4.75
2022-05-23,1.4980,5.1989,1.3526,1.0280,0.84820,135.34,1.4588,1.0577,0.267919,0.182845,...,3.75,2.0,14.0,1.75,0.25,0.50,14.0,0.875,0.00,4.75
2022-05-24,1.4982,5.1623,1.3626,1.0310,0.84783,136.05,1.4639,1.0659,0.269618,0.200358,...,3.75,2.0,14.0,1.75,0.25,0.50,14.0,0.875,0.00,4.75


In [ ]:
from gingado.augmentation import AugmentSDMX
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('augmentation', AugmentSDMX(sources={'BIS': 'WS_CBPOL_D'})),
    ('imp', IterativeImputer(max_iter=10)),
    ('forest', RandomForestRegressor())
], verbose=True)

And since `AugmentSDMX` can be included in a `Pipeline`, it can also be fine-tuned by parameter search techniques (such as grid search), further helping users make the best of available data to enhance performance of their models.

In [ ]:
grid = GridSearchCV(
    estimator=pipeline,
    param_grid={'augmentation': ['passthrough', AugmentSDMX(sources={'ECB': 'CISS'})]},
    verbose=3,
    cv=TimeSeriesSplit()
    )

y_pred_grid = grid.fit(X_train, y_train).predict(X_test)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   0.3s
[CV 1/5] END ..........augmentation=passthrough;, score=0.484 total time=   0.4s
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   0.6s
[CV 2/5] END ..........augmentation=passthrough;, score=0.437 total time=   0.6s
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   0.9s
[CV 3/5] END ..........augmentation=passthrough;, score=0.910 total time=   1.0s
[Pipeline] ...... (step 1 of 3) Processing augmentation, t

2022-05-28 01:10:58,142 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   9.8s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.1s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   0.8s


2022-05-28 01:11:08,678 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...
[CV 1/5] END augmentation=AugmentSDMX(sources={'ECB': 'CISS'});, score=0.557 total time=  15.3s


...

Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...
[CV 4/5] END augmentation=AugmentSDMX(sources={'ECB': 'CISS'});, score=0.928 total time=  40.7s


2022-05-28 01:12:43,088 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=  22.5s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.3s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   5.3s


2022-05-28 01:13:11,231 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...
[CV 5/5] END augmentation=AugmentSDMX(sources={'ECB': 'CISS'});, score=-1.438 total time=  50.3s
[Pipeline] ...... (step 1 of 3) Processing augmentation, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing imp, total=   0.0s
[Pipeline] ............ (step 3 of 3) Processing forest, total=   1.9s


In [ ]:
grid.best_params_

{'augmentation': 'passthrough'}

In [ ]:
print(f"The last value in the training dataset was {y_train.tail(1).to_numpy()}. The predicted value was {y_pred_grid}, and the actual value was {y_test.to_numpy()}.")

The last value in the training dataset was [5.1741]. The predicted value was [5.181137], and the actual value was [5.0959].


## Sources of data

`gingado` seeks to only lists realiable data sources by choice, with a focus on official sources. This is meant to provide users with the trust that their dataset will be complemented by reliable sources. Unfortunately, it is not possible at this stage to include *all* official sources given the substantial manual and maintenance work. `gingado` leverages the existence of the [Statistical Data and Metadata eXchange (SDMX)](https://sdmx.org), an organisation of official data sources that establishes common data and metadata formats, to download data that is relevant (and hopefully also useful) to users.

The function below from the package [simpledmx](https://github.com/dkgaraujo/simpledmx) returns a list of codes corresponding to the data sources available to provide `gingado` users with data through SDMX.

In [ ]:
import pandasdmx as sdmx
sdmx.list_sources()

['ABS',
 'ABS_XML',
 'BBK',
 'BIS',
 'CD2030',
 'ECB',
 'ESTAT',
 'ILO',
 'IMF',
 'INEGI',
 'INSEE',
 'ISTAT',
 'LSD',
 'NB',
 'NBB',
 'OECD',
 'SGR',
 'SPC',
 'STAT_EE',
 'UNICEF',
 'UNSD',
 'WB',
 'WB_WDI']